In [ ]:
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn import tree
from sklearn.preprocessing import LabelEncoder as LB
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('games_clean.csv')
df

In [ ]:
for name in df.columns:
    if df[name].isna().sum() > 0:
        df[name].fillna(df[name].mean(), inplace=True)

In [ ]:
LE = LB()


#df.drop(['RatingsBreakdown-Recommended', 'RatingsBreakdown-Meh', 'RatingsBreakdown-Exceptional', 'RatingsBreakdown-Skip', 'Presence'], inplace=True, axis=1)



class_names = df['Metacritic'].unique()

df.loc[:, 'Metacritic'] = LE.fit_transform(df['Metacritic'])

X = df.loc[:, df.columns != 'Metacritic']
Y = df['Metacritic']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, shuffle=True)

DT = DTC(max_depth=6, criterion='entropy')
DTCfitted = DT.fit(X_train, y_train)
print(df.columns)

In [ ]:
fig = plt.figure(figsize=(40,40), dpi=400)
_ = tree.plot_tree(DTCfitted,
                   feature_names= df.loc[:, df.columns != 'Metacritic'].columns,
                   class_names= class_names,
                   filled=True)

In [ ]:
from sklearn.metrics import accuracy_score, mean_squared_error

In [ ]:
accuracy_score(DTCfitted.predict(X_test), y_test)